# Get the reactions in carveme model into a format that can be uploaded into kbase

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import os
dpath = os.path.join('..','Models')
os.listdir(dpath)

['README.txt', 'HOT1A3_ncbi_nogapfill.xml', 'HOT1A3_ncbi.xml']

In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'mid' : m.id,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return pd.Series(res)

In [5]:
import os
model_dpath = os.path.join('..', 'Models')
model_fname = 'HOT1A3_ncbi.xml'
model_nogap_fname = 'HOT1A3_ncbi_nogapfill.xml'

model_fpath = os.path.join(model_dpath, model_fname)


In [6]:
model = cobra.io.read_sbml_model(model_fpath)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [7]:
model_nogap = cobra.io.read_sbml_model(os.path.join(model_dpath, model_nogap_fname))

In [8]:
model

Name,HOT1A3_ncbi
Memory address,0x07f5944018490
Number of metabolites,1546
Number of reactions,2341
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [9]:
model_nogap

Name,HOT1A3_ncbi_nogapfill
Memory address,0x07f5944018e80
Number of metabolites,1546
Number of reactions,2340
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [10]:
r = model.reactions[25]

In [11]:
r.annotation

{'sbo': 'SBO:0000185',
 'metanetx.reaction': 'MNXR94753',
 'seed.reaction': 'rxn07942'}

In [12]:
r.genes

frozenset()

In [13]:
r

Reaction identifier,2AGPE160tipp
Name,2-Acyl-sn-glycero-3-phosphoethanolamine (n-C16:0) transporter via facilitated diffusion (periplasm)
Memory address,0x7f594398dcd0
Stoichiometry,2agpe160_p --> 2agpe160_c 2-Acyl-sn-glycero-3-phosphoethanolamine (n-C16:0) --> 2-Acyl-sn-glycero-3-phosphoethanolamine (n-C16:0)
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
def _r_to_dict(r):
    res = {
        'reaction': r.name,
        'rid': r.id, 
        'genes' : [g.name for g in r.genes],
        'genes_number' : len(r.genes),
        
    }
    res.update(r.annotation)
    return res
    

rlist = [_r_to_dict(r) for r in model_nogap.reactions]
 

In [16]:
def _r_to_dict(r):
    res = {
        'reaction': r.name,
        'rid': r.id, 
        'genes' : [g.name for g in r.genes],
        'genes_number' : len(r.genes),
        
    }
    res.update(r.annotation)
    return res
    

rgaplist = [_r_to_dict(r) for r in model.reactions]
 

In [17]:
df = pd.DataFrame(rlist)

In [18]:
dfgap = pd.DataFrame(rgaplist)

In [19]:
df

,reaction,rid,genes,genes_number,sbo,metanetx.reaction,biocyc,seed.reaction,ec-code,rhea,kegg.reaction,reactome
0,"1,2 diacylglycerol transport via flipping (per...",12DGR120tipp,[],0,SBO:0000185,MNXR94675,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,2 diacylglycerol transport via flipping (per...",12DGR140tipp,[],0,SBO:0000185,MNXR94676,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,2 diacylglycerol transport via flipping (per...",12DGR141tipp,[],0,SBO:0000185,MNXR94677,NaN,NaN,NaN,NaN,NaN,NaN
3,"1,2 diacylglycerol transport via flipping (per...",12DGR161tipp,[],0,SBO:0000185,MNXR94679,NaN,NaN,NaN,NaN,NaN,NaN
4,"1,2 diacylglycerol transport via flipping (per...",12DGR180tipp,[],0,SBO:0000185,MNXR94680,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2335,R_EX_xtsn_e,EX_xtsn_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2336,R_EX_xyl__D_e,EX_xyl__D_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2337,R_EX_zn2_e,EX_zn2_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2338,Biomass reaction,Growth,[],0,SBO:0000629,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.genes_number.value_counts()

1     1139
0      544
2      390
3      170
4       42
5       35
7        8
8        5
6        5
12       1
13       1
Name: genes_number, dtype: int64

In [21]:
(~df.isna()).sum()

reaction             2340
rid                  2340
genes                2340
genes_number         2340
sbo                  2340
metanetx.reaction    1588
biocyc                805
seed.reaction        1169
ec-code               882
rhea                  864
kegg.reaction         649
reactome              245
dtype: int64

In [22]:
df.columns

Index(['reaction', 'rid', 'genes', 'genes_number', 'sbo', 'metanetx.reaction',
       'biocyc', 'seed.reaction', 'ec-code', 'rhea', 'kegg.reaction',
       'reactome'],
      dtype='object')

In [23]:
(~df.isna()).groupby([ 'seed.reaction', 'ec-code', 'kegg.reaction',]).size().reset_index()

,seed.reaction,ec-code,kegg.reaction,0
0,False,False,False,1040
1,False,False,True,1
2,False,True,False,87
3,False,True,True,43
4,True,False,False,415
5,True,False,True,2
6,True,True,False,149
7,True,True,True,603


In [24]:
(~df.isna()).groupby(['metanetx.reaction', 'seed.reaction', ]).size().reset_index()

,metanetx.reaction,seed.reaction,0
0,False,False,752
1,True,False,419
2,True,True,1169


In [25]:
df.head()

,reaction,rid,genes,genes_number,sbo,metanetx.reaction,biocyc,seed.reaction,ec-code,rhea,kegg.reaction,reactome
0,"1,2 diacylglycerol transport via flipping (per...",12DGR120tipp,[],0,SBO:0000185,MNXR94675,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,2 diacylglycerol transport via flipping (per...",12DGR140tipp,[],0,SBO:0000185,MNXR94676,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,2 diacylglycerol transport via flipping (per...",12DGR141tipp,[],0,SBO:0000185,MNXR94677,NaN,NaN,NaN,NaN,NaN,NaN
3,"1,2 diacylglycerol transport via flipping (per...",12DGR161tipp,[],0,SBO:0000185,MNXR94679,NaN,NaN,NaN,NaN,NaN,NaN
4,"1,2 diacylglycerol transport via flipping (per...",12DGR180tipp,[],0,SBO:0000185,MNXR94680,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.loc[df.genes_number == 0, 'reaction'].unique()

array(['1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C12:0)',
       '1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:0)',
       '1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:1)',
       '1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C16:1)',
       '1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C18:0)',
       '(R)-Propane-1,2-diol transport via diffusion (extracellular to periplasm)',
       '(S)-Propane-1,2-diol transport via diffusion (extracellular to periplasm)',
       '1,4-alpha-D-glucan transport via diffusion (extracellular to periplasm) irreversible',
       '2 5 diketo D gluconate transport via proton symport  reversible  periplasm ',
       '2 5 diketo D gluconate transport via diffusion  extracellular to periplasm ',
       '2-Acyl-sn-glycero-3-phosphatidate (n-C16:0) transporter via facilitated diffusion (periplasm)',
       '2-Acyl-sn-glycero-3-pho

In [27]:
dfgap.loc[~dfgap.rid.isin(df.rid)]

,reaction,rid,genes,genes_number,sbo,metanetx.reaction,biocyc,seed.reaction,ec-code,rhea,kegg.reaction,reactome
2340,Phenylpyruvate oxidation chemical,PHPYROX,[],0,SBO:0000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
model.reactions.PHPYROX

Reaction identifier,PHPYROX
Name,Phenylpyruvate oxidation chemical
Memory address,0x7f5942357fd0
Stoichiometry,o2_c + phpyr_c --> bzal_c + h_c + oxa_c O2 O2 + Phenylpyruvate --> Benzaldehyde + H+ + Oxalate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [29]:
df.rid.unique()

array(['12DGR120tipp', '12DGR140tipp', '12DGR141tipp', ..., 'EX_zn2_e',
       'Growth', 'ATPM'], dtype=object)

In [30]:
jfpath = '../../../../tools/ModelSEEDDatabase/Biochemistry/reactions.json'

In [31]:
import json

In [32]:

with open(jfpath) as user_file:
  parsed_json = json.load(user_file)

In [33]:
parsed_json

[{'abbreviation': 'R00004',
  'abstract_reaction': None,
  'aliases': ['AraCyc: INORGPYROPHOSPHAT-RXN',
   'BiGG: IPP1; PPA; PPA_1; PPAm',
   'BrachyCyc: INORGPYROPHOSPHAT-RXN',
   'KEGG: R00004',
   'MetaCyc: INORGPYROPHOSPHAT-RXN',
   'Name: Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase'],
  'code': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0]',
  'compound_ids': 'cpd00001;cpd00009;cpd00012;cpd00067',
  'definition': '(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] + (1) H+[0]',
  'deltag': -3.46,
  'deltagerr': 0.05,
  'direction': '=',
  'ec_numbers': ['3.6.1.1'],
  'equation': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] + (1) cpd00067[0]',
  'id': 'rxn00001',
  'is_obsolete': 0,
  'is_transport': 0,
  'linked_reaction': 'rxn27946;rxn2794

In [34]:
r = parsed_json[0]

In [35]:
r

{'abbreviation': 'R00004',
 'abstract_reaction': None,
 'aliases': ['AraCyc: INORGPYROPHOSPHAT-RXN',
  'BiGG: IPP1; PPA; PPA_1; PPAm',
  'BrachyCyc: INORGPYROPHOSPHAT-RXN',
  'KEGG: R00004',
  'MetaCyc: INORGPYROPHOSPHAT-RXN',
  'Name: Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase'],
 'code': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0]',
 'compound_ids': 'cpd00001;cpd00009;cpd00012;cpd00067',
 'definition': '(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] + (1) H+[0]',
 'deltag': -3.46,
 'deltagerr': 0.05,
 'direction': '=',
 'ec_numbers': ['3.6.1.1'],
 'equation': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] + (1) cpd00067[0]',
 'id': 'rxn00001',
 'is_obsolete': 0,
 'is_transport': 0,
 'linked_reaction': 'rxn27946;rxn27947;rxn27948;rxn32487;

In [36]:
r['id']

'rxn00001'

In [37]:
r['aliases']

['AraCyc: INORGPYROPHOSPHAT-RXN',
 'BiGG: IPP1; PPA; PPA_1; PPAm',
 'BrachyCyc: INORGPYROPHOSPHAT-RXN',
 'KEGG: R00004',
 'MetaCyc: INORGPYROPHOSPHAT-RXN',
 'Name: Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase']

In [38]:
for a in r['aliases']:
    print(a.split(': '))

['AraCyc', 'INORGPYROPHOSPHAT-RXN']
['BiGG', 'IPP1; PPA; PPA_1; PPAm']
['BrachyCyc', 'INORGPYROPHOSPHAT-RXN']
['KEGG', 'R00004']
['MetaCyc', 'INORGPYROPHOSPHAT-RXN']
['Name', 'Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase']


In [112]:
def _modelseed_to_dict(r):
    res  = {
        'id': r['id'],        
        'is_obsolete' : r['is_obsolete']        
    }
    if 'aliases' in r and r['aliases'] is not None:
        for a in r['aliases']:
            spl = a.split(': ')
            if len(spl) != 2 and spl[0] != 'Name':
                print (spl)
            res[spl[0]] = spl[1].split('; ')
    return res

dfseed = pd.DataFrame([_modelseed_to_dict(r) for r in parsed_json])
    
    

In [113]:
dfseed = dfseed.loc[dfseed.is_obsolete == 0]

In [114]:
dfseed

,id,is_obsolete,AraCyc,BiGG,BrachyCyc,KEGG,MetaCyc,Name,ChlamyCyc,AlgaGEM,...,iAbaylyiv4,iJR904,iMEO21,iNJ661,iSB619,iRR1083,iSO783,iND750,iGT196,iPS189
0,rxn00001,0,[INORGPYROPHOSPHAT-RXN],"[IPP1, PPA, PPA_1, PPAm]",[INORGPYROPHOSPHAT-RXN],[R00004],[INORGPYROPHOSPHAT-RXN],"[Diphosphate phosphohydrolase, Inorganic dipho...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rxn00002,0,[ALLOPHANATE-HYDROLASE-RXN],"[ALPHNH, DUR1_2]",NaN,[R00005],[ALLOPHANATE-HYDROLASE-RXN],"[Allophanate hydrolase, AtzF, TrzF, Urea-1-car...",[ALLOPHANATE-HYDROLASE-RXN],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rxn00003,0,NaN,[ILV2_2],NaN,[R00006],NaN,[2-acetolactate pyruvate-lyase (carboxylating)...,NaN,[R_R00006_c],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rxn00004,0,NaN,NaN,NaN,[R00008],NaN,[4-Hydroxy-4-methyl-2-oxoglutarate pyruvate-ly...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rxn00006,0,[CATAL-RXN],"[CAT, CATp, CTA1, CTT1]",[CATAL-RXN],[R00009],"[CATAL-RXN, RXN-12121]","[Catalase, caperase, catalase A, catalase-pero...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43769,rxn48571,0,NaN,NaN,NaN,[R02164],NaN,[Succinate:ubiquinone oxidoreductase],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43770,rxn48572,0,NaN,NaN,NaN,NaN,NaN,[S-adenosyl-L-methione:demethylmenaquinone met...,NaN,[R_R04993_c],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43771,rxn48573,0,[3.5.1.88-RXN],NaN,[3.5.1.88-RXN],[R05635],[3.5.1.88-RXN],"[3.5.1.88-RXN.d, Formyl-L-methionyl peptide am...",[3.5.1.88-RXN],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43772,rxn48574,0,NaN,NaN,NaN,[R03874],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
dfseed = dfseed.explode('BiGG')

In [116]:
dfseed_cmp = dfseed[['id','BiGG']].drop_duplicates(subset=['id','BiGG']).dropna(subset='BiGG')

In [117]:
df

,reaction,rid,genes,genes_number,sbo,metanetx.reaction,biocyc,seed.reaction,ec-code,rhea,kegg.reaction,reactome
0,"1,2 diacylglycerol transport via flipping (per...",12DGR120tipp,[],0,SBO:0000185,MNXR94675,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,2 diacylglycerol transport via flipping (per...",12DGR140tipp,[],0,SBO:0000185,MNXR94676,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,2 diacylglycerol transport via flipping (per...",12DGR141tipp,[],0,SBO:0000185,MNXR94677,NaN,NaN,NaN,NaN,NaN,NaN
3,"1,2 diacylglycerol transport via flipping (per...",12DGR161tipp,[],0,SBO:0000185,MNXR94679,NaN,NaN,NaN,NaN,NaN,NaN
4,"1,2 diacylglycerol transport via flipping (per...",12DGR180tipp,[],0,SBO:0000185,MNXR94680,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2335,R_EX_xtsn_e,EX_xtsn_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2336,R_EX_xyl__D_e,EX_xyl__D_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2337,R_EX_zn2_e,EX_zn2_e,[],0,SBO:0000627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2338,Biomass reaction,Growth,[],0,SBO:0000629,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
dfmerge = pd.merge(df, dfseed_cmp, left_on='rid', right_on='BiGG', how='left' )

In [119]:
dfmerge.BiGG.isna().sum()

983

In [120]:
dfmerge = dfmerge.explode('genes')

In [121]:
dfmerge.columns

Index(['reaction', 'rid', 'genes', 'genes_number', 'sbo', 'metanetx.reaction',
       'biocyc', 'seed.reaction', 'ec-code', 'rhea', 'kegg.reaction',
       'reactome', 'id', 'BiGG'],
      dtype='object')

In [122]:
dfmerge.loc[dfmerge.genes_number >0, ['rid', 'id']].drop_duplicates()

,rid,id
6,12PPDRtpp,rxn07919
8,12PPDStpp,rxn07921
9,13PPDH,NaN
10,13PPDH2,NaN
11,13PPDH2_1,NaN
...,...,...
2136,ZNabcpp,rxn05150
2137,Zn2tex,rxn09390
2138,ilvg,NaN
2139,r0191,NaN


In [123]:
dfmerge.genes.unique()

array([nan, 'G_WP_049588165_1', 'G_WP_061439201_1', ...,
       'G_WP_061485725_1', 'G_WP_014979112_1', 'G_WP_061486830_1'],
      dtype=object)

In [124]:
dfgene2id = dfmerge[['genes', 'id']].dropna()

In [125]:
dfgene2id['protein_id'] = dfgene2id['genes'].str.replace(r'^G_', '', regex=True).str.replace(r'_1$', '.1', regex=True)

In [126]:
mapdf = pd.read_csv(os.path.join(model_dpath, 'data','HOT1A3_NCBI_gene_list.csv'))

In [127]:
mapdf

,locus_tag,type,gene,gene_biotype,old_locus_tag,product,protein_id
0,ACZ81_RS00010,gene,dnaN,protein_coding,ACZ81_00010,DNA polymerase III subunit beta,WP_012516526.1
1,ACZ81_RS00015,gene,recF,protein_coding,ACZ81_00015,DNA replication/repair protein RecF,WP_061438857.1
2,ACZ81_RS00020,gene,gyrB,protein_coding,ACZ81_00020,DNA topoisomerase (ATP-hydrolyzing) subunit B,WP_014977848.1
3,ACZ81_RS00025,gene,NaN,protein_coding,ACZ81_00025,hypothetical protein,WP_039234509.1
4,ACZ81_RS00030,gene,glyS,protein_coding,ACZ81_00030,glycine--tRNA ligase subunit beta,WP_061438859.1
...,...,...,...,...,...,...,...
4171,ACZ81_RS20715,pseudogene,NaN,pseudogene,ACZ81_20715,cupredoxin domain-containing protein,NaN
4172,ACZ81_RS20720,gene,NaN,protein_coding,ACZ81_20720,DUF2933 domain-containing protein,WP_012516909.1
4173,ACZ81_RS20725,pseudogene,NaN,pseudogene,ACZ81_20725,isoprenylcysteine carboxylmethyltransferase fa...,NaN
4174,ACZ81_RS20730,gene,NaN,protein_coding,ACZ81_20730,hypothetical protein,WP_012516911.1


In [128]:
dfgene2id = pd.merge(dfgene2id, mapdf[['locus_tag', 'protein_id']], how='left', on='protein_id')

In [129]:
dfgene2id1 =  dfgene2id[['locus_tag', 'id']].dropna().drop_duplicates()

In [130]:
dfgene2id.loc[dfgene2id.locus_tag.isna()].genes.unique()

array(['G_spontaneous'], dtype=object)

In [131]:
dfgene2id1.locus_tag.value_counts()

ACZ81_RS18370    40
ACZ81_RS19805    40
ACZ81_RS09690    29
ACZ81_RS06240    29
ACZ81_RS13410    26
                 ..
ACZ81_RS02395     1
ACZ81_RS17550     1
ACZ81_RS02370     1
ACZ81_RS14480     1
ACZ81_RS19410     1
Name: locus_tag, Length: 867, dtype: int64

In [132]:
dfgene2id1.to_csv(os.path.join(model_dpath, 'data','carveme_gene2reaction.tsv'), sep='\t', header=False, index=False)

In [141]:
dfgene2id.loc[dfgene2id.locus_tag.isin(['ACZ81_RS18370'])]

,genes,id,protein_id,locus_tag
162,G_WP_014951106_1,rxn08063,WP_014951106.1,ACZ81_RS18370
411,G_WP_014951106_1,rxn08184,WP_014951106.1,ACZ81_RS18370
414,G_WP_014951106_1,rxn08186,WP_014951106.1,ACZ81_RS18370
426,G_WP_014951106_1,rxn08197,WP_014951106.1,ACZ81_RS18370
450,G_WP_014951106_1,rxn08234,WP_014951106.1,ACZ81_RS18370
455,G_WP_014951106_1,rxn05467,WP_014951106.1,ACZ81_RS18370
460,G_WP_014951106_1,rxn08241,WP_014951106.1,ACZ81_RS18370
471,G_WP_014951106_1,rxn05212,WP_014951106.1,ACZ81_RS18370
647,G_WP_014951106_1,rxn08428,WP_014951106.1,ACZ81_RS18370
735,G_WP_014951106_1,rxn08475,WP_014951106.1,ACZ81_RS18370


In [142]:
dfmerge.loc[dfmerge.genes.isin(['G_WP_014951106_1'])]

,reaction,rid,genes,genes_number,sbo,metanetx.reaction,biocyc,seed.reaction,ec-code,rhea,kegg.reaction,reactome,id,BiGG
64,3-methylbutanoic acid transport via diffusion ...,3MBtex,G_WP_014951106_1,2,SBO:0000185,MNXR94918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,4-hydroxybenzaldehyde transport via diffusion ...,4HBALDtex,G_WP_014951106_1,2,SBO:0000185,MNXR95006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,4-hydroxy-benzyl alcohol transport via diffusi...,4HBAtex,G_WP_014951106_1,2,SBO:0000185,MNXR95007,META:TRANS-RXN0-274,rxn12376,NaN,NaN,NaN,NaN,NaN,NaN
75,4-Hydroxybenzoate transport (extracellular),4HBZtex,G_WP_014951106_1,2,SBO:0000185,MNXR95013,NaN,"[rxn09792, rxn12787]",NaN,NaN,NaN,NaN,NaN,NaN
242,Acetate transport via diffusion (extracellular...,ACtex,G_WP_014951106_1,2,SBO:0000185,MNXR95431,"[META:RXN0-1981, META:TRANS-RXN0-567]","[rxn08063, rxn09787, rxn09866, rxn10904]",NaN,"[27814, 27815, 27816, 27817]",NaN,NaN,rxn08063,ACtex
450,Arsenate transporter (extracellular to periplasm),ASO4tex,G_WP_014951106_1,2,SBO:0000185,MNXR96076,NaN,rxn05506,NaN,NaN,NaN,NaN,NaN,NaN
490,Butyrate transport via diffusion (extracellula...,BUTtex,G_WP_014951106_1,2,SBO:0000185,MNXR96352,NaN,NaN,NaN,NaN,NaN,NaN,rxn08184,BUTtex
494,Benzoate transport via diffusion (extracellula...,BZtex,G_WP_014951106_1,2,SBO:0000185,MNXR96370,NaN,rxn12436,NaN,"[32811, 32812, 32813, 32814]",NaN,NaN,NaN,NaN
496,Calcium (Ca+2) transport via diffusion (extrac...,CA2tex,G_WP_014951106_1,2,SBO:0000185,MNXR96436,"[META:TRANS-RXN-144, META:TRANS-RXN0-497]",rxn08186,3.6.3.8,"[29671, 29672, 29673, 29674]",NaN,"[R-ATH-2685505, R-ATH-418309, R-BTA-139854, R-...",rxn08186,CA2tex
510,Cadmium (Cd+2) transport via diffusion (extrac...,CD2tex,G_WP_014951106_1,2,SBO:0000185,MNXR96499,META:RXN0-10,"[rxn08197, rxn08198]",NaN,"[28707, 28708, 28709, 28710]",NaN,NaN,rxn08197,CD2tex


In [135]:
mapdf.loc[mapdf.protein_id.isin(['WP_014950043.1'])]

,locus_tag,type,gene,gene_biotype,old_locus_tag,product,protein_id
2565,ACZ81_RS12780,gene,adk,protein_coding,ACZ81_12770,adenylate kinase,WP_014950043.1


In [136]:
a,b = model.reactions.Zn2tex.genes

In [137]:
b

Gene identifier,WP_014951106_1
Name,G_WP_014951106_1
Memory address,0x7f5943aae430
Functional,True
In 53 reaction(s),"NI2tex, 4HBALDtex, CA2tex, GALURtex, LEUtex, COBALT2tex, PItex, VALtex, LYStex, OXFOtex, CD2tex, GUAtex, FORtex, Ktex, PTAtex, ILEtex, 3MBtex, PPAtex, H2Otex, MOBDtex, 4HBZtex, BZtex, FE3tex,..."


In [138]:
a

Gene identifier,WP_014951442_1
Name,G_WP_014951442_1
Memory address,0x7f5943aae400
Functional,True
In 53 reaction(s),"NI2tex, 4HBALDtex, CA2tex, GALURtex, LEUtex, COBALT2tex, PItex, VALtex, LYStex, OXFOtex, CD2tex, GUAtex, FORtex, Ktex, PTAtex, ILEtex, 3MBtex, PPAtex, H2Otex, MOBDtex, 4HBZtex, BZtex, FE3tex,..."


In [139]:
dfseed.is_obsolete.value_counts()

0    38760
Name: is_obsolete, dtype: int64

In [143]:
dfseed.loc[dfseed.id.isin(['rxn14090'])]

,id,is_obsolete,AraCyc,BiGG,BrachyCyc,KEGG,MetaCyc,Name,ChlamyCyc,AlgaGEM,...,iAbaylyiv4,iJR904,iMEO21,iNJ661,iSB619,iRR1083,iSO783,iND750,iGT196,iPS189
12922,rxn14090,0,NaN,NaN,NaN,[R08952],[2.5.1.71-RXN],"[LG 2''-dimethylallyltransferase, LGDT, dimeth...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
